In [1]:
import pandas as pd
import numpy as np
import requests
from urllib.parse import urlencode


import datetime

pd.set_option('display.max_columns', 80)
pd.set_option('display.max_rows', 5)

In [2]:
# Read in the data
df_original = pd.read_csv('redfin.csv')
df_original.shape

(24, 16)

In [3]:
df = df_original.copy()
df

,sold_date,address,city,state,zip,price,lot_size,acres,price_per_acre,redfin_url,mls,lat,long,status,parcel_number,regrid_url
0,NaN,Wild Wings Ln,Thurmond,NC,28683,"$63,000.00",4919231,112.930005,$557.87,https://www.redfin.com/NC/Thurmond/Wild-Wings-...,NaN,36.395656,-80.848901,Not Reviewed,NaN,NaN
1,NaN,Westridge Dr,Mount Airy,NC,27030,"$85,000.00",3985740,91.500000,$928.96,https://www.redfin.com/NC/Mount-Airy/Westridge...,NaN,36.524123,-80.668731,Not Reviewed,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22,NaN,245 Brentons Ridge Trl,State Road,NC,28676,"$200,000.00",1089000,25.000000,"$8,000.00",https://www.redfin.com/NC/State-Road/245-Brent...,NaN,36.352267,-80.881297,Not Reviewed,NaN,NaN
23,March-30-2023,288 E Collins Ln,Dobson,NC,27017,"$200,000.00",1010156,23.189991,"$8,624.41",https://www.redfin.com/NC/Dobson/288-E-Collins...,1071633.0,36.396048,-80.699611,Not Reviewed,NaN,NaN


In [4]:
def get_redgrid_data(lat, long):
    base_url = "https://app.regrid.com/search.json"
    params = {
        'query': f"{lat} {long}",
        'autocomplete': '1',
        'context': '/us/nc/surry',
        'strict': 'false'
    }
    # print('computed params', params)

    url = base_url + '?' + urlencode(params)
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
    }

    response = requests.get(url, headers=headers)
    data = response.json()
    return data


def extract_values(data):
    # check if data is empty
    if len(data) == 0:
      return [None, None]
    return [data[0]['parcelnumb'], data[0]['path']]

data = get_redgrid_data(36.395656, -80.848901)
parcel_number, redgrid_path = extract_values(data)
print(parcel_number, redgrid_path)

495600581103 /us/nc/surry/bryan/13964


In [5]:
for index, row in df.iterrows():
    latitude = df.at[index,'lat']
    longitude = df.at[index,'long'] 

    data = get_redgrid_data(latitude, longitude)
    parcel_number, redgrid_path = extract_values(data)

    df.at[index, 'parcel_number'] = parcel_number
    df.at[index, 'redgrid_path'] = redgrid_path


df

,sold_date,address,city,state,zip,price,lot_size,acres,price_per_acre,redfin_url,mls,lat,long,status,parcel_number,regrid_url,redgrid_path
0,NaN,Wild Wings Ln,Thurmond,NC,28683,"$63,000.00",4919231,112.930005,$557.87,https://www.redfin.com/NC/Thurmond/Wild-Wings-...,NaN,36.395656,-80.848901,Not Reviewed,495600581103,NaN,/us/nc/surry/bryan/13964
1,NaN,Westridge Dr,Mount Airy,NC,27030,"$85,000.00",3985740,91.500000,$928.96,https://www.redfin.com/NC/Mount-Airy/Westridge...,NaN,36.524123,-80.668731,Not Reviewed,500104938439,NaN,/us/nc/surry/stewarts-creek/45230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22,NaN,245 Brentons Ridge Trl,State Road,NC,28676,"$200,000.00",1089000,25.000000,"$8,000.00",https://www.redfin.com/NC/State-Road/245-Brent...,NaN,36.352267,-80.881297,Not Reviewed,494500528582,NaN,/us/nc/surry/bryan/16902
23,March-30-2023,288 E Collins Ln,Dobson,NC,27017,"$200,000.00",1010156,23.189991,"$8,624.41",https://www.redfin.com/NC/Dobson/288-E-Collins...,1071633.0,36.396048,-80.699611,Not Reviewed,499600974002,NaN,/us/nc/surry/dobson/11093


In [6]:

for index, row in df.iterrows():
    redgrid_path = df.at[index,'redgrid_path']
    if redgrid_path is not None:
      # path value is like: /us/nc/surry/bryan/13964
      # converts it to:
      # https://app.regrid.com/us/nc/surry#b=admin&base=satellite&p=/us/nc/surry/bryan/13964&t=property
        df.at[index, 'redgrid_url'] = f"https://app.regrid.com{redgrid_path}#b=admin&base=satellite&p={redgrid_path}&t=property"


In [7]:
df.to_csv('redfin_with_redgrid.csv', index=False)